In [21]:
import numpy as np  # importing required libraries
from sklearn import datasets,svm # we'll use a dataset from the datasets in sklearn

In [5]:
iris = datasets.load_iris() #using iris dataset


In [7]:
import pandas as pd           #importing pandas to convert dataset into dataframe
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower']=iris.target    #making target as new column in dataframe
df['flower']=df['flower'].apply(lambda x: iris.target_names[x]) #putting names of the flower in flower column


In [8]:
df[40:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
40,5.0,3.5,1.3,0.3,setosa
41,4.5,2.3,1.3,0.3,setosa
42,4.4,3.2,1.3,0.2,setosa
43,5.0,3.5,1.6,0.6,setosa
44,5.1,3.8,1.9,0.4,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [15]:
from sklearn.model_selection import train_test_split,cross_val_score #spitting dataframe  into two parts
X_train,X_test,Y_train,Y_test = train_test_split(iris.data,iris.target,test_size=0.3)

In [16]:
model = svm.SVC(kernel='rbf',C=30,gamma = 'auto') #using Support Vector Machine model to train data
model.fit(X_train,Y_train)
model.score(X_test,Y_test)  #checking the score on test data

0.9333333333333333

USE K FOLD CROSS VALIDATION

In [18]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5).mean() 
#viewing cross validation score with manually selecting parameters

0.9733333333333334

In [19]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5).mean()

0.9800000000000001

In [22]:
#printing scores with changing parameters
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}

for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [32]:
from sklearn.model_selection import GridSearchCV #Using GridSearchCV for concise code
cl = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
cl.fit(iris.data, iris.target)
cl.cv_results_
dir(cl)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_in_',
 'n_jobs

In [ ]:
df1 = pd.DataFrame(cl.cv_results_)
df1.head()

In [27]:
df1[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [29]:
cl.best_params_ #Finding out the best parameter according to GridSearchCV

{'C': 1, 'kernel': 'rbf'}

In [31]:
cl.best_score_ 

0.9800000000000001

In [33]:
from sklearn.model_selection import RandomizedSearchCV #Selecting parameters randomly using RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,rbf,0.98
1,1,linear,0.98


In [34]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Keeping models in the form of keys in a dictionary and their respective model and parameters
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [36]:
#printing model with respective scores and best parameters
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
